In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [2]:

# Let's first load and examine the content of the uploaded files to understand their structure.
import pandas as pd

# Load the transaction example file
txn_example_path = './DataSources/Copy of AT txn example - AT txn example.csv'
txn_df = pd.read_csv(txn_example_path)

# Load the MCC codes file
mcc_codes_path = './DataSources/mcc_codes.csv'
mcc_df = pd.read_csv(mcc_codes_path)

# Display the first few rows of each dataframe to understand their structure
txn_df.head(), mcc_df.head()

(  Transaction Date Value Date Statement Code Reference  Debit Amount  \
 0        04-Jan-24        NaN            POS       BAT         93.74   
 1        23-Dec-23        NaN            POS       BAT         20.19   
 2        22-Dec-23        NaN            POS       BAT        580.23   
 3        22-Dec-23        NaN            POS       BAT         11.98   
 4        21-Dec-23        NaN            POS       BAT         27.55   
 
    Credit Amount                       Merchant Name  \
 0            NaN  MY MUSCLE CHEF PTY LTD YE NN 01JAN   
 1            NaN  GIGA                   68 25 21DEC   
 2            NaN  FLYSCOOT.0000000K5NQRV SI NG 20DEC   
 3            NaN  GOOGLE*YOUTUBEPREMIUM  G. CO 20DEC   
 4            NaN  CHATGPT SUBSCRIPTION   HT TP 20DEC   
 
                      Category  MCC CODE           Additional Reference  \
 0          Food & Restaurants       NaN  4628-4500-5183-2831 AUD100.55   
 1  Telecommunication Services    4814.0            4628-4500-5183

In [3]:
# Approach:
# We will try to find the most relevant MCC code for each merchant by searching for keywords in the MCC descriptions.
# Given the complexity and potential variability of merchant names, this process might not cover all cases accurately.

# Simplify the MCC descriptions by combining them into a single column for easier searching
mcc_df['description'] = mcc_df[['edited_description', 'combined_description', 'usda_description', 'irs_description']].fillna('').apply(lambda x: ' '.join(x), axis=1)

# Create a function to search for the most relevant MCC code based on merchant name keywords
def find_mcc_category(merchant_name):
    # Prepare the merchant name for keyword search
    keywords = merchant_name.upper().split()
    # Initialize variables to store the best match MCC and its score
    best_match = None
    best_score = 0
    # Iterate through each MCC description to find the best match
    for _, row in mcc_df.iterrows():
        score = sum(keyword in row['description'].upper() for keyword in keywords)
        if score > best_score:
            best_score = score
            best_match = row['irs_description']  # Assuming IRS description as the final category for simplicity
    return best_match

# Apply the function to the merchant names in the transaction dataframe
txn_df['Generated Category'] = txn_df['Merchant Name'].apply(find_mcc_category)

# Display the updated dataframe with generated categories
txn_df[['Merchant Name', 'Category', 'Generated Category']].head()


,Merchant Name,Category,Generated Category
0,MY MUSCLE CHEF PTY LTD YE NN 01JAN,Food & Restaurants,Hotels/Motels/Inns/Resorts
1,GIGA 68 25 21DEC,Telecommunication Services,None
2,FLYSCOOT.0000000K5NQRV SI NG 20DEC,Travel,"Roofing/Siding, Sheet Metal"
3,GOOGLE*YOUTUBEPREMIUM G. CO 20DEC,NaN,Agricultural Cooperative
4,CHATGPT SUBSCRIPTION HT TP 20DEC,NaN,Airlines


In [5]:
txn_df

,Transaction Date,Value Date,Statement Code,Reference,Debit Amount,Credit Amount,Merchant Name,Category,MCC CODE,Additional Reference,Misc Reference,Generated Category
0,04-Jan-24,NaN,POS,BAT,93.74,NaN,MY MUSCLE CHEF PTY LTD YE NN 01JAN,Food & Restaurants,NaN,4628-4500-5183-2831 AUD100.55,NaN,Hotels/Motels/Inns/Resorts
1,23-Dec-23,NaN,POS,BAT,20.19,NaN,GIGA 68 25 21DEC,Telecommunication Services,4814.0,4628-4500-5183-2831,NaN,None
2,22-Dec-23,NaN,POS,BAT,580.23,NaN,FLYSCOOT.0000000K5NQRV SI NG 20DEC,Travel,4511.0,4628-4500-5183-2831,NaN,"Roofing/Siding, Sheet Metal"
3,22-Dec-23,NaN,POS,BAT,11.98,NaN,GOOGLE*YOUTUBEPREMIUM G. CO 20DEC,NaN,NaN,4628-4500-5183-2831,NaN,Agricultural Cooperative
4,21-Dec-23,NaN,POS,BAT,27.55,NaN,CHATGPT SUBSCRIPTION HT TP 20DEC,NaN,NaN,4628-4500-5183-2831 USD20.00,NaN,Airlines
...,...,...,...,...,...,...,...,...,...,...,...,...
204,10-Oct-23,NaN,POS,BAT,11.60,NaN,GRAB* A-5H2WM5IWWE3A SI NG 07OCT,NaN,NaN,4628-4500-5183-2831,NaN,"Roofing/Siding, Sheet Metal"
205,10-Oct-23,NaN,POS,BAT,55.00,NaN,FEATHERZ STUDIO SI NG 07OCT,NaN,NaN,4628-4500-5183-2831,NaN,"Roofing/Siding, Sheet Metal"
206,08-Oct-23,08-Oct-23,POS,BAT,38.10,NaN,DELIVEROO SI NG 06OCT,NaN,NaN,4628-4500-5183-2831,NaN,"Roofing/Siding, Sheet Metal"
207,06-Oct-23,NaN,POS,BAT,19.20,NaN,GOPAY-GOJEK SI NG 03OCT,NaN,NaN,4628-4500-5183-2831,NaN,"Roofing/Siding, Sheet Metal"
